# Descripción del Notebook

## Objetivo del Notebook

Procesar y corregir los errores en los datos de canciones.

## Pasos Realizados

1. **Importación de Librerías**:
    - Importamos las librerías necesarias (`pandas`, `ipywidgets`).

2. **Cargar Datos**:
    - Cargamos los datos de canciones con errores desde un archivo CSV.
    - Verificamos si existen correcciones previas y las cargamos.

3. **Filtrar Datos**:
    - Filtramos las canciones que aún no han sido corregidas.

4. **Interfaz de Corrección**:
    - Utilizamos `ipywidgets` para crear una interfaz interactiva que permite seleccionar una canción, ver sus detalles y corregir errores en la URL de Spotify y las letras.

5. **Guardar Correcciones**:
    - Guardamos las correcciones realizadas en un nuevo archivo CSV.

## Conclusión

Este notebook proporciona una herramienta interactiva para corregir errores en los datos de canciones, asegurando que los datos estén limpios y listos para su uso posterior.

In [9]:
import pandas as pd

# 📌 Rutas de los archivos
errores_file = r"C:\Users\solan\Downloads\get_data_from_songs\src\errores_canciones.csv"
correcciones_file = r"C:\Users\solan\Downloads\get_data_from_songs\src\errores_canciones_corregidos.csv"

# 📌 Cargar errores
df_errores = pd.read_csv(errores_file)

# 📌 Verificar si ya existe un archivo con correcciones previas
try:
    df_corregidos = pd.read_csv(correcciones_file)
    corregidos_ids = set(df_corregidos["recording_id"])
    print(f"📌 Se encontraron {len(corregidos_ids)} correcciones previas.")
except FileNotFoundError:
    df_corregidos = pd.DataFrame(columns=df_errores.columns)
    corregidos_ids = set()

# 📌 Filtrar solo las canciones aún no corregidas
df_pendientes = df_errores[~df_errores["recording_id"].isin(corregidos_ids)]

# 📌 Mostrar las primeras filas de los errores
df_pendientes.head(10)


📌 Se encontraron 1 correcciones previas.


,artist_name,song_name,recording_id,danceable,not_danceable,male,female,timbre_bright,timbre_dark,tonal,...,spotify_url,album_name,album_release_date,duration_ms,popularity,language,playlist_ids,playlists_names,combined_genres,processed_lyrics


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# 📌 Función para corregir errores con más contexto
def corregir_errores():
    global df_pendientes, df_corregidos  # Asegura acceso a los DataFrames

    # 📌 Dropdown para seleccionar canción por ID
    dropdown_id = widgets.Dropdown(
        options=[(f"{row['song_name']} - {row['artist_name']} ({row['recording_id']})", row["recording_id"]) 
                 for _, row in df_pendientes.iterrows()],
        description="Canción:",
        disabled=False,
        layout=widgets.Layout(width="80%")
    )

    # 📌 Cuadro para mostrar detalles de la canción seleccionada
    song_info = widgets.Output()

    def update_song_info(change):
        with song_info:
            clear_output()
            song_id = change.new
            selected_song = df_pendientes[df_pendientes["recording_id"] == song_id].iloc[0]
            print(f"🎵 **Canción:** {selected_song['song_name']}")
            print(f"🎤 **Artista:** {selected_song['artist_name']}")
            print(f"🔗 **URL Actual:** {selected_song['spotify_url']}")
            print("\n📜 **Letra Original (Primeros 500 caracteres):**")
            print(selected_song['processed_lyrics'][:500] + "..." if isinstance(selected_song['processed_lyrics'], str) else "Letra no disponible")

    dropdown_id.observe(update_song_info, names="value")

    input_url = widgets.Text(
        description="Nueva URL:",
        placeholder="Introduce la URL corregida (opcional)"
    )

    input_lyrics = widgets.Textarea(
        description="Letra corregida:",
        placeholder="Introduce la letra corregida",
        layout=widgets.Layout(width='100%', height='200px')
    )

    save_button = widgets.Button(description="Guardar Corrección", button_style="success")
    output = widgets.Output()

    def save_correction(b):
        with output:
            output.clear_output()
            song_id = dropdown_id.value
            new_url = input_url.value.strip()
            new_lyrics = input_lyrics.value.strip()

            # 📌 Eliminar saltos de línea en la letra corregida
            new_lyrics = " ".join(new_lyrics.splitlines())

            # 📌 Obtener la fila de la canción seleccionada
            row = df_pendientes[df_pendientes["recording_id"] == song_id].iloc[0]

            # 📌 Aplicar correcciones si el usuario ingresó algo
            if new_url:
                row["spotify_url"] = new_url
            if new_lyrics:
                row["processed_lyrics"] = new_lyrics  # 🔥 Se guarda sin saltos de línea

            # 📌 Guardar en el DataFrame corregido
            global df_corregidos
            df_corregidos = pd.concat([df_corregidos, row.to_frame().T], ignore_index=True)

            # 📌 Guardar progresivamente para no perder cambios
            df_corregidos.to_csv(correcciones_file, index=False)

            print(f"✅ Canción '{row['song_name']}' corregida y guardada sin saltos de línea.")

    save_button.on_click(save_correction)

    display(dropdown_id, song_info, input_url, input_lyrics, save_button, output)

# 🔥 Ejecutar la función para comenzar
corregir_errores()


Dropdown(description='Canción:', layout=Layout(width='80%'), options=(('death graduates - the octopus project …

Output()

Text(value='', description='Nueva URL:', placeholder='Introduce la URL corregida (opcional)')

Textarea(value='', description='Letra corregida:', layout=Layout(height='200px', width='100%'), placeholder='I…

Button(button_style='success', description='Guardar Corrección', style=ButtonStyle())

Output()